In [34]:
import requests
import json
import time
import pandas
from operator import itemgetter
# variables 
# patch note
Patch = "12.1.1"
# minimum games played for a champ win rate 
MinimumGamesPlayed = 1
# specify champ name to get most buy items
Champion = "Renekton"
# objectiv you wanna win rate when took first
Obj = "inhibitor"
# did they win the game ?
GameWin = True
# did they took it first ?
FirstObj = True
date = "1641373200"
 

In [12]:
# Recupere la list des chall
api_key = ""
GetChallPlayers = "https://euw1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5" + "?api_key=" + api_key
response1 = requests.get(GetChallPlayers).json()


In [13]:
# recupere le puuid des chall
print (time.strftime("%H : %M : %S"))
info = (response1['entries'])
playersPuuid = []
playersName = []
for i in range(299):
        GetPlayersPuuid = "https://euw1.api.riotgames.com/lol/summoner/v4/summoners/" + info[i]['summonerId'] + "?api_key=" + api_key
        response2 = requests.get(GetPlayersPuuid).json()
        playersPuuid.append(response2["puuid"])
        time.sleep(2)
print("end")
print (time.strftime("%H : %M : %S"))
# recupere les 20 last games de chaque joueur depuis une certaine date
print (time.strftime("%H : %M : %S"))
PlayersGames = []
for i in range(299):
        GetPlayersGames = "https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/" + playersPuuid[i] + "/ids?startTime=" + date + "&queue=420&type=ranked&start=0&count=20&api_key=" + api_key
        response2 = requests.get(GetPlayersGames).json()
        PlayersGames = PlayersGames + response2
        time.sleep(2)
print("end")
print (time.strftime("%H : %M : %S"))
PlayersGames.sort()
# verifie que la liste des games a ete trie
print(len(PlayersGames))
PlayersGames = list(set(PlayersGames))
seen = set()
result = []
for item in PlayersGames:
    if item not in seen:
        seen.add(item)
        result.append(item)
print(len(result))
# recupere les infos de chaque game
print (time.strftime("%H : %M : %S"))
GamesInfo = []
PlayersGamesRange = len(PlayersGames) - 1
for i in range(PlayersGamesRange):
    GetGamesInfo = "https://europe.api.riotgames.com/lol/match/v5/matches/" + PlayersGames[i] + "?api_key=" + api_key
    response3 = requests.get(GetGamesInfo).json()
    GamesInfo.append(response3)
    time.sleep(2)
print("end")
print (time.strftime("%H : %M : %S"))


10 : 28 : 17
end
10 : 38 : 52
10 : 38 : 52
end
10 : 49 : 27
281
135
10 : 49 : 27
end
10 : 54 : 16


In [35]:
# nb de games par champions
# recupere la liste des champions joue sur toutes les games dispo, et on tri par ordre alphabet
from operator import itemgetter
champsList = []
count = len(GamesInfo) - 1
for j in range(count):
    for i in range(10):
        champsList.append(GamesInfo[j]["info"]["participants"][i]["championName"])
# compte le nombre d apparition d un champion
ChampWinList = [[x, champsList.count(x)] for x in set(champsList)]
ChampPickedCounter = [[x, champsList.count(x)] for x in set(champsList)]
# pour trier par nb de games : , key = itemgetter(1), reverse = True
ChampPickedCounter = sorted(ChampPickedCounter)
ChampWinList = sorted(ChampWinList)
# nb de loose par champions
# recupere la liste des champions joue sur toutes les games dispo, et on tri par ordre alphabet
ChampLoseList = []
count = len(GamesInfo) - 1
for j in range(count):
    for i in range(10):
        if GamesInfo[j]["info"]["participants"][i]["win"] is False:
            ChampLoseList.append(GamesInfo[j]["info"]["participants"][i]["championName"])
# compte le nombre d apparition d un champion
ChampLoseListCounter = [[x, ChampLoseList.count(x)] for x in set(ChampLoseList)]
# pour trier par nb de games : , key = itemgetter(1), reverse = True
ChampLoseListCounter = sorted(ChampLoseListCounter)
# nb de games loose par champions
champWinRate = []
countLose = len(ChampLoseListCounter) - 1
countPick = len(ChampPickedCounter) - 1
# calcul du winrate d un champion
for i in range(countPick):
    for j in range(countLose):
        if ChampWinList[i][0] == ChampLoseListCounter[j][0]:
            ChampWinList[i][1] = ChampWinList[i][1] - ChampLoseListCounter[j][1]
            winrate = round(ChampWinList[i][1] * 100 / ChampPickedCounter[i][1], 2)
            if ChampPickedCounter[i][1] > MinimumGamesPlayed:
                champWinRate.append([ChampWinList[i][0], winrate])
champWinRate = sorted(champWinRate, key = lambda x: x[1], reverse = True)


In [36]:
# on affiche toutes nos donnees

print("------------Picks-----------------")
print(sorted(ChampPickedCounter, key=lambda x: x[1], reverse=True))
print("-------------Win rate----------------")
print(champWinRate)
print("------------win nb-----------------")
print(ChampWinList)
print("-------------lose nb----------------")
print(ChampLoseListCounter)

------------Picks-----------------
[['Jhin', 38], ['LeeSin', 38], ['Caitlyn', 36], ['Diana', 36], ['Yasuo', 36], ['Jayce', 35], ['Karma', 30], ['Jinx', 28], ['Lux', 28], ['Viktor', 27], ['Vex', 26], ['Talon', 25], ['Ezreal', 23], ['Kaisa', 23], ['Gangplank', 21], ['Irelia', 21], ['Sylas', 21], ['Thresh', 21], ['Draven', 19], ['Yone', 19], ['Fiora', 18], ['Graves', 18], ['Qiyana', 18], ['Zed', 18], ['Akali', 17], ['Viego', 16], ['XinZhao', 16], ['Nautilus', 15], ['Pyke', 15], ['Akshan', 14], ['Corki', 14], ['Ekko', 14], ['Leblanc', 14], ['Leona', 14], ['Lulu', 14], ['Gragas', 13], ['Janna', 13], ['Rakan', 13], ['Renekton', 13], ['Soraka', 13], ['Syndra', 13], ['Aphelios', 12], ['Kayn', 12], ['Nunu', 12], ['Tryndamere', 12], ['Vayne', 12], ['Camille', 11], ['Karthus', 11], ['Katarina', 11], ['Rell', 11], ['Lucian', 10], ['Poppy', 10], ['Sona', 10], ['Blitzcrank', 9], ['Hecarim', 9], ['Jax', 9], ['Nidalee', 9], ['Taric', 9], ['Xayah', 9], ['Ashe', 8], ['Nami', 8], ['Riven', 8], ['Veigar',

In [28]:
# Calcul du nombre d'occurrences des items pour un champion donne
ItemList = []
WinOrNot = True
for e in range(count):
    for i in range(10):
        for j in range(6):
            if GamesInfo[e]["info"]["participants"][i]["championName"] == Champion:
                # est ce que c est une game win ou non (peut etre mis en commit)
                # if GamesInfo[e]["info"]["participants"][i]["win"] == TrueOrFalse:
                    ItemList.append(GamesInfo[e]["info"]["participants"][i]["item" + str(j)])
print(ItemList)
ItemListcounter = [[x, ItemList.count(x)] for x in set(ItemList)]
ItemListcounter = sorted(ItemListcounter, key = lambda x: x[1], reverse = True)

# on change l'alias des items pour leur nom
GetItemInfo = requests.get('https://ddragon.leagueoflegends.com/cdn/' + Patch + '/data/en_US/item.json').json()
for i in range(len(ItemListcounter)):
    if ItemListcounter[i][0] != 0:
        number = str(ItemListcounter[i][0])
        ItemListcounter[i][0] = GetItemInfo["data"][str(number)]["name"]
    if ItemListcounter[i][0] == 0:
        ItemListcounter[i][0] = "null"
print(ItemListcounter)


[3053, 6333, 6630, 3047, 1038, 3071, 1038, 2421, 6630, 2055, 3047, 3053, 1055, 3047, 6630, 3123, 0, 0, 1037, 6630, 2031, 3047, 1055, 1036, 3047, 0, 6630, 1055, 3044, 1028, 3047, 6693, 3067, 1054, 3211, 3053, 1055, 3053, 6630, 3111, 1038, 3123, 3053, 6630, 3111, 3133, 1054, 1029, 1054, 6631, 3053, 1033, 3047, 0, 6630, 6333, 3153, 3071, 1033, 3047, 6630, 3053, 8001, 3047, 3026, 1054, 1028, 6630, 3047, 1037, 1036, 1054, 3053, 2421, 6630, 3211, 3082, 3047]
[['Goredrinker', 11], ['Plated Steelcaps', 11], ["Sterak's Gage", 8], ["Doran's Shield", 5], ['null', 4], ["Doran's Blade", 4], ['B. F. Sword', 3], ['Ruby Crystal', 2], ['Null-Magic Mantle', 2], ["Spectre's Cowl", 2], ['Long Sword', 2], ['Pickaxe', 2], ["Mercury's Treads", 2], ["Executioner's Calling", 2], ["Death's Dance", 2], ['Broken Stopwatch', 2], ['Black Cleaver', 2], ['Cloth Armor', 1], ['Control Ward', 1], ["Warden's Mail", 1], ["Prowler's Claw", 1], ["Caulfield's Warhammer", 1], ["Anathema's Chains", 1], ['Blade of The Ruined Ki

In [19]:
# calcul du winrate par rapport au premier objectif
GamesFirstObjWin = 0
for i in range(len(GamesInfo)):
    for e in range(2):
        if GamesInfo[i]["info"]["teams"][e]["win"] == GameWin and GamesInfo[i]["info"]["teams"][e]["objectives"][Obj]["first"] == FirstObj:
            GamesFirstObjWin = GamesFirstObjWin + 1
GamesFirstObjWinRate = GamesFirstObjWin * 100 / len(GamesInfo)
GamesFirstObjWinRate = round(GamesFirstObjWinRate, 2)
print("Winrate with", Obj, "killed first", GamesFirstObjWinRate)


Winrate with inhibitor killed first 71.64


In [20]:
# creation d un dictionaire avec la key et le nom du champion
bannedchamp = []
GetChampName = requests.get('https://ddragon.leagueoflegends.com/cdn/' + Patch + '/data/en_US/champion.json').json()
ChampNameList = []
for k in GetChampName["data"]:
    ChampNameList.append(k)
ChampListKey = []
for i in range(len(ChampNameList)):
    ChampListKey.append([ChampNameList[i], GetChampName["data"][ChampNameList[i]]["key"]])

# calcul du nombre de ban d un champion
for f in range(len(GamesInfo)):
    for e in range(2):
        for i in range(5):
            if GamesInfo[f]["info"]["teams"][e]["bans"][i]["championId"] != -1:
                bannedchamp.append(GamesInfo[f]["info"]["teams"][e]["bans"][i]["championId"])
bannedchamp = [[x, bannedchamp.count(x)] for x in set(bannedchamp)]
bannedchamp = sorted(bannedchamp, key = lambda x: x[1], reverse = True)

# on change la key par le nom du champion
for i in range(len(bannedchamp)):
    for e in range(len(ChampListKey)):
        if str(bannedchamp[i][0]) == ChampListKey[e][1]:
            bannedchamp[i][0] = ChampListKey[e][0]
print("Most banned champ :", bannedchamp)


Most banned champ : [['Akshan', 99], ['Talon', 73], ['Yuumi', 64], ['Kassadin', 56], ['Draven', 56], ['Qiyana', 46], ['Irelia', 41], ['Lulu', 40], ['Yone', 40], ['Jax', 39], ['Viego', 39], ['LeeSin', 35], ['Leblanc', 33], ['Diana', 26], ['Viktor', 25], ['Vex', 23], ['Fiora', 22], ['Evelynn', 21], ['Caitlyn', 20], ['Jayce', 20], ['Tryndamere', 19], ['Sona', 19], ['Jinx', 18], ['Zed', 17], ['Katarina', 16], ['Akali', 16], ['Shaco', 14], ['Vayne', 14], ['Bard', 14], ['Lux', 13], ['Rengar', 13], ['Camille', 13], ['Thresh', 13], ['Kayn', 11], ['Anivia', 9], ['Janna', 9], ['Pyke', 9], ['Nidalee', 9], ['Nautilus', 8], ['Kaisa', 7], ['TwistedFate', 6], ['Sylas', 6], ['Singed', 6], ['XinZhao', 5], ['Karma', 5], ['Ezreal', 5], ['Riven', 5], ['Lucian', 5], ['Soraka', 4], ['Karthus', 4], ['Gangplank', 4], ['Nocturne', 4], ['Graves', 4], ['Yasuo', 4], ['Ekko', 4], ['Zilean', 3], ['Malphite', 3], ['Leona', 3], ['Zoe', 3], ['TahmKench', 3], ['Nami', 3], ['Nunu', 2], ['Corki', 2], ['Trundle', 2], ['El

In [21]:
WinBlueCount = 0

LoseBlueCount = 0

for i in range(len(GamesInfo)):
    if GamesInfo[i]["info"]["teams"][0]["win"] is True:
        WinBlueCount = WinBlueCount + 1
    elif GamesInfo[i]["info"]["teams"][0]["win"] is False:
        LoseBlueCount = LoseBlueCount + 1

print("Blue side win : ", WinBlueCount)

print("Blue side lose : ", LoseBlueCount)


Blue side win :  72
Blue side lose :  62


In [22]:
# Average game time
GameTime = []
AverageTime = 0
for i in range(len(GamesInfo)):
    GameTime.append(GamesInfo[i]["info"]["gameDuration"])
AverageTime = round(sum(GameTime) / len(GameTime), 2)
AverageTimeInMinute = time.strftime("%M:%S", time.localtime(AverageTime))

# calcul du gain d exp et de gold des supports
SupportGoldEarned = []
SupportExp = []
AverageSupportGold = 0
AverageSupportExp = 0
for i in range(len(GamesInfo)):
    for e in range(10):
        if GamesInfo[i]["info"]["participants"][e]["role"] == "SUPPORT":
            SupportGoldEarned.append(GamesInfo[i]["info"]["participants"][e]["goldEarned"])
AverageSupportGold = sum(SupportGoldEarned) / len(SupportGoldEarned)
AverageSupportGold = round(AverageSupportGold, 2)
AverageSupportGoldMin = round((AverageSupportGold / AverageTime) * 60, 2)

for i in range(len(GamesInfo)):
    for e in range(10):
        if GamesInfo[i]["info"]["participants"][e]["role"] == "SUPPORT":
            SupportExp.append(GamesInfo[i]["info"]["participants"][e]["champExperience"])
AverageSupportExp = sum(SupportExp) / len(SupportExp)
AverageSupportExp = round(AverageSupportExp, 2)
AverageSupportExpMin = round((AverageSupportExp / AverageTime) * 60, 2)

# calcul du gain d exp et de gold des jungler
JunglerGoldEarned = []
JunglerExp = []
AverageJunglerExp = 0
AverageJunglerGold = 0
for i in range(len(GamesInfo)):
    for e in range(10):
        if GamesInfo[i]["info"]["participants"][e]["individualPosition"] == "JUNGLE":
            JunglerGoldEarned.append(GamesInfo[i]["info"]["participants"][e]["goldEarned"])
AverageJunglerGold = sum(JunglerGoldEarned) / len(JunglerGoldEarned)
AverageJunglerGold = round(AverageJunglerGold, 2)
AverageJunglerGoldMin = round((AverageJunglerGold / AverageTime) * 60, 2)

for i in range(len(GamesInfo)):
    for e in range(10):
        if GamesInfo[i]["info"]["participants"][e]["individualPosition"] == "JUNGLE":
            JunglerExp.append(GamesInfo[i]["info"]["participants"][e]["champExperience"])
AverageJunglerExp = sum(JunglerExp) / len(JunglerExp)
AverageJunglerExp = round(AverageJunglerExp, 2)
AverageJunglerExpMin = round((AverageJunglerExp / AverageTime) * 60, 2)
# calcul du gain d exp et de gold des adc
CarryGoldEarned = []
CarryExp = []
AverageCarryExp = 0
AverageGoldExp = 0
for i in range(len(GamesInfo)):
    for e in range(10):
        if GamesInfo[i]["info"]["participants"][e]["role"] == "CARRY":
            CarryGoldEarned.append(GamesInfo[i]["info"]["participants"][e]["goldEarned"])
AverageCarryGold = sum(CarryGoldEarned) / len(CarryGoldEarned)
AverageCarryGold = round(AverageCarryGold, 2)
AverageCarryGoldMin = round((AverageCarryGold / AverageTime) * 60, 2)

for i in range(len(GamesInfo)):
    for e in range(10):
        if GamesInfo[i]["info"]["participants"][e]["role"] == "CARRY":
            CarryExp.append(GamesInfo[i]["info"]["participants"][e]["champExperience"])
AverageCarryExp = sum(CarryExp) / len(CarryExp)
AverageCarryExp = round(AverageCarryExp, 2)
AverageCarryExpMin = round((AverageCarryExp / AverageTime) * 60, 2)

# calcul du gain d exp et de gold des mid
MidGoldEarned = []
MidExp = []
AverageMidExp = 0
AverageMidGold = 0
for i in range(len(GamesInfo)):
    for e in range(10):
        if GamesInfo[i]["info"]["participants"][e]["individualPosition"] == "MIDDLE":
            MidGoldEarned.append(GamesInfo[i]["info"]["participants"][e]["goldEarned"])
AverageMidGold = sum(MidGoldEarned) / len(MidGoldEarned)
AverageMidGold = round(AverageMidGold, 2)
AverageMidGoldMin = round((AverageMidGold / AverageTime) * 60, 2)

for i in range(len(GamesInfo)):
    for e in range(10):
        if GamesInfo[i]["info"]["participants"][e]["individualPosition"] == "MIDDLE":
            MidExp.append(GamesInfo[i]["info"]["participants"][e]["champExperience"])
AverageMidExp = sum(MidExp) / len(MidExp)
AverageMidExp = round(AverageMidExp, 2)
AverageMidExpMin = round((AverageMidExp / AverageTime) * 60, 2)

# calcul du gain d exp et de gold des top
TopGoldEarned = []
TopExp = []
AverageTopExp = 0
AverageTopGold = 0
for i in range(len(GamesInfo)):
    for e in range(10):
        if GamesInfo[i]["info"]["participants"][e]["individualPosition"] == "TOP":
            TopGoldEarned.append(GamesInfo[i]["info"]["participants"][e]["goldEarned"])
AverageTopGold = sum(TopGoldEarned) / len(TopGoldEarned)
AverageTopGold = round(AverageTopGold, 2)
AverageTopGoldMin = round((AverageTopGold / AverageTime) * 60, 2)

for i in range(len(GamesInfo)):
    for e in range(10):
        if GamesInfo[i]["info"]["participants"][e]["individualPosition"] == "TOP":
            TopExp.append(GamesInfo[i]["info"]["participants"][e]["champExperience"])
AverageTopExp = sum(TopExp) / len(TopExp)
AverageTopExp = round(AverageTopExp, 2)
AverageTopExpMin = round((AverageTopExp / AverageTime) * 60, 2)


In [23]:
print(AverageTopGoldMin)
print(AverageJunglerGoldMin)
print(AverageMidGoldMin)
print(AverageCarryGoldMin)
print(AverageSupportGoldMin)

431.98
448.54
434.39
473.52
295.33
